# Туристическая карта города Муром


**Дата**: 14.05.2025

**Курс**: Methods of Spatial Analysis. Advanced Level. // HSE, Moscow, spring 2025

**Состав команды**: Дубова Анастасия, Полянская Татьяна, Нефедова Ксения

## 1. Установка и импорт библиотек


In [1]:
pip install folium

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install geometry

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install shapely

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install geopandas==0.10.2

Note: you may need to restart the kernel to use updated packages.


In [5]:
import pandas as pd
import folium
from shapely import geometry
import geopandas as gpd


## 2. Чтение базового слоя


In [6]:
# в качестве базового и первого тематического слоя по настройке карты выбран слой с заведениями общепита

data_food = gpd.read_file('/Users/macbook/Desktop/gis pro/project 3 stage 2/food_utm.geojson')
data_food.head()

,fid,Название,Регион,Район,Район_1,Город,Район города,Адрес,Индекс,Телефон,...,youtube,twitter,skype,icq,googleplus,linkedin,pinterest,Широта,Долгота,geometry
0,1,Империя,Владимирская область,Муром городской округ,Муром городской округ,Муром,,"Воровского, 24",,"+7 (49234) 7‒74‒31, +7 (49234) 7‒74‒34",...,,,,,,,,55.57662,42.055252,POINT (314369.293 6162896.531)
1,2,Лаванда,Владимирская область,Муром городской округ,Муром городской округ,Муром,,"Коммунистическая, 31",,,...,,,,,,,,55.579003,42.044573,POINT (313707.615 6163190.215)
2,3,Ташир пицца,Владимирская область,Муром городской округ,Муром городской округ,Муром,,"улица Куликова, 7а",,8‒800‒200‒59‒10,...,,,,,,,,55.571097,42.037197,POINT (313205.281 6162330.603)
3,4,Милано,Владимирская область,Муром городской округ,Муром городской округ,Муром,,"улица Ленина, 32",,+7 (49234) 7‒79‒39,...,,,,,,,,55.577238,42.054245,POINT (314308.752 6162967.969)
4,5,Хуторок,Владимирская область,Муром городской округ,Муром городской округ,Муром,,"посёлок Механизаторов, 51",,+7 (49234) 7‒13‒39,...,,,,,,,,55.595657,42.034234,POINT (313135.296 6165070.533)


## 3. Создание веб-карты


In [7]:
# перепроецирование слоя для работы с folium

data_food = data_food.to_crs("EPSG:4326")

In [8]:
# настройка карты

m = folium.Map(location=[data_food.centroid.y.mean(), data_food.centroid.x.mean()], zoom_start=12, tiles="cartodb positron", control_scale=True)

/var/folders/gn/plhm4x3x6g5411wxtnvnrdgr0000gn/T/ipykernel_14849/4146167550.py:3: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  m = folium.Map(location=[data_food.centroid.y.mean(), data_food.centroid.x.mean()], zoom_start=12, tiles="cartodb positron", control_scale=True)
/var/folders/gn/plhm4x3x6g5411wxtnvnrdgr0000gn/T/ipykernel_14849/4146167550.py:3: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  m = folium.Map(location=[data_food.centroid.y.mean(), data_food.centroid.x.mean()], zoom_start=12, tiles="cartodb positron", control_scale=True)


In [9]:
# создание слоев в карте

import folium.plugins
from folium.plugins import GroupedLayerControl

mcg = folium.FeatureGroup(name="туристическая карта города Муром")
m.add_child(mcg)

g0 = folium.plugins.FeatureGroupSubGroup(mcg, "граница города")
m.add_child(g0)

g1 = folium.plugins.FeatureGroupSubGroup(mcg, "категории объектов")
m.add_child(g1)

g2 = folium.plugins.FeatureGroupSubGroup(mcg, "тепловая карта")
m.add_child(g2)

g3 = folium.plugins.FeatureGroupSubGroup(mcg, "отели")
m.add_child(g3)

g4 = folium.plugins.FeatureGroupSubGroup(mcg, "религиозные объекты")
m.add_child(g4)

g5 = folium.plugins.FeatureGroupSubGroup(mcg, "туристический маршрут «По следам Илья Муромца»")
m.add_child(g5)

folium.LayerControl(collapsed=False).add_to(m)

# визуализация общепита представлена двумя способами, поэтому слои для общественного питания сгруппированы
# допускается одновременное подключение двух слоев группы

GroupedLayerControl(
    groups={'общественное питание': [g1, g2]},
    collapsed=False,
    exclusive_groups=False
).add_to(m)

m

In [10]:
# добавление слоя с границей города

data_border = gpd.read_file('/Users/macbook/Desktop/gis pro/project 3 stage 2/border.geojson')

geojson_data = data_border.__geo_interface__
folium.GeoJson(
    geojson_data,
    style_function=lambda feature: {
        'fillColor': 'transparent',  
        'color': 'black',   
        'weight': 1,     
    }
).add_to(g0)

In [11]:
# первый способ визуализации слоя с общепитом - иконки
# иконки имет цвет, отражающий категорию объекта (бар, ресторан, кафе и т.д.)
# при нажатии на иконку отображается категория и название заведения

geo_data_food_list = [[point.xy[1][0], point.xy[0][0]] for point in data_food.geometry]

i = 0
for coordinates in geo_data_food_list:
    if data_food['Подрубрика'][i] == "Бар":
        type_color = "gray"
    elif data_food['Подрубрика'][i] == "Кафе":
        type_color = "lightblue"
    elif data_food['Подрубрика'][i] == "Быстрое питание":
        type_color = "lightred"
    elif data_food['Подрубрика'][i] == "Ресторан":
        type_color = "beige"
    else:
        type_color = "lightgray"
    g1.add_child( 
        folium.Marker(
            location=coordinates,
            popup="Тип: "
            + str(data_food['Подрубрика'][i])
            + "<br>"
            + "Имя: "
            + str(data_food['Название'][i])
            + "<br>",
            icon=folium.Icon(icon='coffee', prefix='fa',color="%s" % type_color)
        )
    )
    i = i + 1
m

In [12]:
# второй способ визуализации слоя с общепитом - тепловая карта

import numpy as np
import folium
from folium.plugins import HeatMap
heat_data = [[point.xy[1][0], point.xy[0][0]] for point in data_food.geometry]
HeatMap(heat_data, radius=30, gradient={0.2: 'white', 0.4: 'yellow', 0.6: 'orange', 0.8: 'red', 1: 'brown'}).add_to(g2)
m

In [13]:
# второй тематический слой - слой с отелями и гостиницами города 
# визуализация слоя - картодиаграмма с дифференциацией по средней стоимости проживания за ночь
# размер маркера изменяется линейно в зависимости от цены
# при нажатии на круг отображается стоимость проживания и название заведения
# информация также дублируется при наведении мыши

data_hotels = gpd.read_file('/Users/macbook/Desktop/gis pro/project 3 stage 2/hotel.geojson') 

min_price = data_hotels['money'].min()
max_price = data_hotels['money'].max()


for index, row in data_hotels.iterrows():
    price = row['money']

    radius = 2 + (price - min_price) / (max_price - min_price)*18

    folium.CircleMarker(
        location=[row.geometry.y, row.geometry.x],
        radius=radius,
        color="purple",  
        fill=True,
        fill_color="purple",
        fill_opacity=0.7,
        popup=f"Название: {row['name']}, цена за ночь: {price} руб.",  
        tooltip=f"{row['name']}, {price} руб."
    ).add_to(g3)

m


In [14]:
# третий тематический слой - религиозные объекты города 
# для визуализации слоя выбраны кластеры, которые при приближении распадаюся на иконки 
# цвет иконки отображает принадлежность к религии
# при нажатии на иконку отображается религиозная принадлежность и название объекта

from folium.plugins import MarkerCluster
data_religion = gpd.read_file('/Users/macbook/Desktop/gis pro/project 3 stage 2/religion_pnt.geojson') 

data_religion = data_religion.to_crs("EPSG:4326")

geo_data_religion_list = [[point.xy[1][0], point.xy[0][0]] for point in data_religion.geometry]

marker_cluster = MarkerCluster(data = [geo_data_religion_list], name = 'cluster')
i = 0
for coordinates in geo_data_religion_list:
    if data_religion['religion'][i] == "christian":
        type_color = "lightblue"
    elif data_religion['religion'][i] == "muslim":
        type_color = "gray"
    else:
        type_color = "lightgray",
    marker_cluster.add_child( 
        folium.Marker(
            location=coordinates,
            clustered_marker = True,
            popup="Тип: "
            + str(data_religion['religion_type'][i])
            + "<br>"
            + "Имя: "
            + str(data_religion['name'][i])
            + "<br>",
            icon=folium.Icon(icon='place-of-worship', prefix='fa',color="%s" % type_color)
        )
    )
    i = i + 1
marker_cluster.add_to(g4)
m

In [15]:
# четвертый тематический слой - туристический маршрут «По следам Ильи Муромца»
# иконки для слоя имеют форму круга, цвет отображает категорию объекта
# иконки соединены маршрутом с указанием стороны движения 
# при нажатии на иконку отображается категория объекта, его название и адрес

from folium.plugins import AntPath, BeautifyIcon

data_file = '/Users/macbook/Desktop/gis pro/project 3 stage 2/route.geojson'

route_data = gpd.read_file(data_file)  

locations = route_data[['lat', 'lon']].values.tolist()

ant_path = AntPath(locations=locations,  reverse=False, color="mediumpurple", weight=5, pulse_color="moccasin")
ant_path.add_to(g5)

for index, row in route_data.iterrows():
    icon = None
    if row['type'] == 'музей':
        icon = folium.plugins.BeautifyIcon(icon="museum", border_color="lightblue", icon_shape="circle")
    elif row['type'] == 'памятник':
        icon = folium.plugins.BeautifyIcon(icon="monument", border_color="lightcoral", icon_shape="circle")
    elif row['type'] == 'кафе':
        icon = folium.plugins.BeautifyIcon(icon="coffee", border_color="wheat", icon_shape="circle")
    elif row['type'] == 'монастырь':
        icon = folium.plugins.BeautifyIcon(icon="church", border_color="gray", icon_shape="circle")
    elif row['type'] == 'часовня':
        icon = folium.plugins.BeautifyIcon(icon="church", border_color="gray", icon_shape="circle")
    else:
        icon = folium.Icon(icon="info-sign", color="blue")  # Иконка по умолчанию

    folium.Marker(
        location=[row['lat'], row['lon']],
        icon=icon,
        popup=f"Название: {row['name']}<br>Тип: {row['type']}<br>Адрес: {row['adress']}",
        tooltip=f"{row['name']}"
    ).add_to(g5)
m

## 4. Дополнительный функционал и сохранение


In [16]:
# добавление курсора мыши

from folium.plugins import MousePosition
MousePosition().add_to(m)
m

In [17]:
# добавление возможности расширения карты на полный экран

from folium.plugins import Fullscreen
folium.plugins.Fullscreen(
    position="topright",
    title="Expand me",
    title_cancel="Exit me",
    force_separate_button=True,
).add_to(m)

m

In [18]:
# сохранение карты

m.save("index.html")